In [1]:
% packages
import pyaudio
import struct
import numpy as np
import matplot.pyplot as plot
import time
from tkinter import TclError

# use this backend to display in separate Tk window
%matplotlib tk

# constants
sampsPerFrame = 1024 * 2             # samples per frame
bytePerSamp = pyaudio.paInt16     # audio format (bytes per sample?)
singMicChan = 1                 # single channel for microphone
sampsPerSec = 44100                 # samples per second


UsageError: Line magic function `%` not found.


In [ ]:
# create matplotlib figure and axes
fig, ax = plt.subplots(1, figsize=(15, 7))

# pyaudio class instance
p = pyaudio.PyAudio()

# stream object to get data from microphone
stream = p.open(
    format=bytePerSamp,
    channels=singMicChan,
    rate=sampsPerSec,
    input=True,
    output=True,
    frames_per_buffer=sampsPerFrame
)

# variable for plotting
x = np.arange(0, 2 * sampsPerFrame, 2)

# create a line object with random data
line, = ax.plot(x, np.random.rand(sampsPerFrame), '-', lw=2)

# basic formatting for the axes
ax.set_title('AUDIO WAVEFORM')
ax.set_xlabel('samples')
ax.set_ylabel('volume')
ax.set_ylim(0, 255)
ax.set_xlim(0, 2 * sampsPerFrame)
plt.setp(ax, xticks=[0, sampsPerFrame, 2 * sampsPerFrame], yticks=[0, 128, 255])

# show the plot
plt.show(block=False)

print('stream started')

# for measuring frame rate
frame_count = 0
start_time = time.time()

while True:
    
    # binary data
    data = stream.read(sampsPerFrame)  
    
    # convert data to integers, make np array, then offset it by 127
    data_int = struct.unpack(str(2 * sampsPerFrame) + 'B', data)
    
    # create np array and offset by 128
    data_np = np.array(data_int, dtype='b')[::2] + 128
    
    line.set_ydata(data_np)
    
    # update figure canvas
    try:
        fig.canvas.draw()
        fig.canvas.flush_events()
        frame_count += 1
        
    except TclError:
        
        # calculate average frame rate
        frame_rate = frame_count / (time.time() - start_time)
        
        print('stream stopped')
        print('average frame rate = {:.0f} FPS'.format(frame_rate))
        break